# LAB 9

Write a local-search algorithm (eg. an EA) able to solve the Problem instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

In [16]:
from abc import abstractmethod


class AbstractProblem:
    def __init__(self):
        self._calls = 0

    @property
    @abstractmethod
    def x(self):
        pass

    @property
    def calls(self):
        return self._calls

    @staticmethod
    def onemax(genome):
        return sum(bool(g) for g in genome)

    def __call__(self, genome):
        self._calls += 1
        fitnesses = sorted((AbstractProblem.onemax(genome[s :: self.x]) for s in range(self.x)), reverse=True)
        val = sum(f for f in fitnesses if f == fitnesses[0]) - sum(
            f * (0.1 ** (k + 1)) for k, f in enumerate(f for f in fitnesses if f < fitnesses[0])
        )
        return val / len(genome)


def make_problem(a):
    class Problem(AbstractProblem):
        @property
        @abstractmethod
        def x(self):
            return a

    return Problem()

In [17]:
from random import choices

fitness = make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10110101101001011111110011100000010000101111110011: 31.56%
11000101100010011101101110100010010011100111111101: 15.33%
00100101100110111100001101110101000010000110001000: 15.33%
10011011110010001010001011010010110110100111100010: 15.33%
11001111011110101101010010010110110011011011001101: 19.11%
00100011101111010111011110000001000101011000001001: 23.56%
01100110000100110011100101110100010111111001010011: 9.13%
10010000000101011110101110010111001100101111010011: 15.33%
01101110001111111110100111010110010011000000010110: 15.33%
11001000111010011000010010000100011010001101100011: 7.33%
10


## Set of useful functions
They are used later on for the creation of the problem, the initialization of the population and consequently for the implementation of the evoluitonary algorithm. Note that different approaches would be used, in order to try different combinations of parameters.

In [18]:
import numpy as np
import random
def create_problem_instance(number):
  return make_problem(number)

def initialize_genomes_population(genome_size, population_size):
  return [choices([0, 1], k=population_size) for _ in range(genome_size)]

def selection1(population, fitness):
    zipped = list(zip(population, fitness))
    sorted_population = sorted(zipped, key=lambda x: x[1], reverse=True)
    return [x[0] for x in sorted_population[:len(population)//2]]

def  selection2(population, fitness,  size):
    sorted_population = sorted(zip(population, fitness), key=lambda x: x[1], reverse=True)
    return [genome for genome, _ in sorted_population[: size]]

def tournament_selection(population, fitnesses, tournament_size):
    selected = []
    for _ in range(2):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament = [(population[i], fitnesses[i]) for i in tournament_indices]
        selected.append(max(tournament, key=lambda x: x[1])[0])
    return selected

def select_best_individual(population, fitness_scores):
  best_individual = population[fitness_scores.index(max(fitness_scores))]
  return best_individual

def crossover(genome1, genome2): # 2 children
    crossover_point1 = np.random.randint(0, len(genome1))

    child1_genome = (np.concatenate((genome1[:crossover_point1], genome2[crossover_point1:])))
    child2_genome = (np.concatenate((genome2[:crossover_point1], genome1[crossover_point1:])))

    return child1_genome, child2_genome

def crossover_one_child(genome1, genome2, crossover_rate):
    if random.random() < crossover_rate:
        point = random.uniform(0, 1)
        crossover_point = int(point * len(genome1))
        child = genome1[:crossover_point] + genome2[crossover_point:]
        return child
    else:
        return genome1

def one_cut_crossover(genome1, genome2):
    crossover_point = random.randint(0,len(genome1)-1)
    child = genome1[:crossover_point] + genome2[crossover_point:]
    return child


def mutate(genome, mutation_rate):
    for i in range(len(genome)):
        if random.random() < mutation_rate:
            genome[i] = 1 - genome[i]
    return genome

def reproduce(selected):
    new_population = []
    # if len(selected) // SELECTION_FACTOR > 1:
    #   top_percent = selected[:int(len(selected)//SELECTION_FACTOR)]
    # else:
    #   top_percent = selected
    top_percent = selected[:len(selected)//SELECTION_FACTOR]
    while len(new_population)  < len(selected):
      parent1 = random.choice(top_percent)
      parent2 = random.choice(top_percent)
      if random.random() < MUTATION_RATE:
          genome1 = mutate(parent1, MUTATION_RATE)
          genome2 = mutate(parent2, MUTATION_RATE)
          new_population.append([genome1])
          new_population.append([genome2])

      elif random.random()> MUTATION_RATE and random.random() < MUTATION_AND_CROSSOVER_RATE:
          genome1 = mutate(parent1, MUTATION_RATE)
          genome2 = mutate(parent2, MUTATION_RATE)
          child1, child2 = crossover(genome1 , genome2)
          new_population.append([child1])
          new_population.append([child2])

      elif random.random() > MUTATION_AND_CROSSOVER_RATE:
            child1, child2 = crossover(parent1 , parent2)
            new_population.append([child1])
            new_population.append([child2])
    return new_population

def replacement(population, new_population, fitness_scores):
     half = selection1(population, fitness_scores)

     res = half + new_population
     return res



## Definition of the parameters and algorithm implementation


Basically, the algorithm tries different approaches to evaluate also which is the best for each problem instance. Here the pseudocode of all the considered options (one generation):



### option 1
```
select the best individuals of size TOURNAMENT SIZE from the population with the function ' selection2'
select 2 parents from the best individuals with function 'tournament_selection'
if a random number is < xover rate:
    generate 1 child with 'crossover_one_child' function
if a random number is < mutation rate:
    apply the 'mutate' function to this child
now the new population is composed of the best individuals + the child
```



### option 2

```
select the half of the population ordere by fitness with the function 'selection1'
select a certain percentage of individuals from this half
select 2 parents from this percentage
(start of 'reproduce' function -->) generate a random number
if this random number < xover rate:
    apply 'crossover' function to produce 2 children
elif this random number is < mutation and xover rate:
    apply both the xover and the mutation to produce 2 children
else
    apply just the 'mutate' function to produce 1 child
(start of 'replacement' function -->) the new population is composed of the selected individuals + the generated children
```

### option 3

```
selecte the best indidivuals with the function ' selection2'
select 2 parents from this elite with 'tournament_selection'
apply one_cut_crossover to produce 1 child
if a random number is < mutation rate:
    apply also the 'mutate' function to this child
the new population is composed of the elite + 1 child
```



### option 4

```
select the best individuals with the function ' selection2'
select the parents with 'tournament_selection'
if a random number is < xover rate:
    apply 'one_cut_crossover' function
    apply also the 'mutate' function
else
    do nothing
the new population is composed of the elite + (eventually) 1 child
```



Further options can be explored and evalutated.

In [19]:
# Parameters
GENOME_LENGTH = 1000
MUTATION_AND_CROSSOVER_RATE = 0.25
SELECTION_FACTOR = 5 # -> in order to take the 20% of the population ordered by fitness



# Parameter ranges
problem_instances = [1, 2, 5, 10]
max_generations = [10, 50, 100]
population_sizes = [50, 100, 200]
crossover_rates = [0.5, 0.7, 0.9]
tournament_sizes = [3, 5, 7, 10, 49]
mutation_rates = [0.001, 0.1, 0.1, 0.15]
offspring_sizes = [20, 50, 100, 200]

# Random choices of the parameters at the starting of the algorithm
MUTATION_RATE = random.choice(mutation_rates)
CROSSOVER_RATE = random.choice(crossover_rates)
POPULATION_SIZE = random.choice(population_sizes)
TOURNAMENT_SIZE = random.choice(tournament_sizes)
MAX_GENERATIONS = random.choice(max_generations)
OFFSPRING_SIZE = random.choice(offspring_sizes)

print('Parameters for this run:')
print(f'mutation rate={MUTATION_RATE}, crossover rate={CROSSOVER_RATE}, population size={POPULATION_SIZE}, tournament size={TOURNAMENT_SIZE}, max number of generations={MAX_GENERATIONS}, offspring size={OFFSPRING_SIZE}')

# Start of the algorithm

for problem_instance_number in problem_instances:
  highest_fitness_per_instance = {}
  highest_fitness_per_instance[problem_instance_number] = (0, None)
  fitness = create_problem_instance(problem_instance_number)
  print(f'PROBLEM INSTANCE NUMBER = {problem_instance_number}:')

  population = initialize_genomes_population(POPULATION_SIZE, GENOME_LENGTH)
  fitness_cache = {}

  # option 1
  def option1(population, fitness_values):
    bests1 = []
    selected =  selection2(population, fitness_values, TOURNAMENT_SIZE)
    new_population = selected.copy()
    while len(new_population) < POPULATION_SIZE:
      parent1, parent2 = tournament_selection(population, fitness_values, TOURNAMENT_SIZE)
      child = crossover_one_child(parent1, parent2, CROSSOVER_RATE)
      child = mutate(child, MUTATION_RATE)
      new_population.extend([child])
      final_population = new_population[:POPULATION_SIZE]

    best_fitness = max(fitness_values)
    best_genome = select_best_individual(final_population, fitness_values)
    bests1.append((best_genome, best_fitness, fitness.calls))

    return bests1

  # option 2
  def option2(population, fitness_values):
    bests1 = []
    selected = selection1(population, fitness_values)
    new_population = reproduce(selected)
    final_population = replacement(population, new_population, fitness_values)

    best_fitness = max(fitness_values)
    best_genome = select_best_individual(final_population, fitness_values)
    bests1.append((best_genome, best_fitness, fitness.calls))

    return bests1

  # option 3
  def option3(population, fitness_values):
    bests1 = []
    selected =  selection2(population, fitness_values, TOURNAMENT_SIZE)
    new_population = selected.copy()
    while len(new_population) < POPULATION_SIZE:
      parent1, parent2 = tournament_selection(population, fitness_values, TOURNAMENT_SIZE)
      child = one_cut_crossover(parent1, parent2)
      child = mutate(child, MUTATION_RATE)
      new_population.extend([child])
      final_population = new_population[:POPULATION_SIZE]

    best_fitness = max(fitness_values)
    best_genome = select_best_individual(final_population, fitness_values)
    bests1.append((best_genome, best_fitness, fitness.calls))

    return bests1

  # option 4
  def option4(population, fitness_values):
    bests1 = []
    elite =  selection2(population, fitness_values, TOURNAMENT_SIZE)
    new_population = elite.copy()
    child = None
    while len(new_population) < POPULATION_SIZE:
      parent1, parent2 = tournament_selection(population, fitness_values, TOURNAMENT_SIZE)
      if random.random() < CROSSOVER_RATE:
        child = one_cut_crossover(parent1, parent2)
        child = mutate(child, MUTATION_RATE)
      if child:
        new_population.extend([child])
      final_population = new_population[:POPULATION_SIZE]

    best_fitness = max(fitness_values)
    best_genome = select_best_individual(final_population, fitness_values)
    bests1.append((best_genome, best_fitness, fitness.calls))

    return bests1

  # # option 5 -> da rivedere
  # def option5(population, fitness_values):
  #   bests1 = []
  #   elite =  selection2(population, fitness_values, TOURNAMENT_SIZE)
  #   new_population =elite.copy()
  #   while len(new_population) < POPULATION_SIZE:
  #     new_population = reproduce(elite)
  #     final_population = replacement(population, new_population, fitness_values)

  #   best_fitness = max(fitness_values)
  #   best_genome = select_best_individual(final_population, fitness_values)
  #   bests1.append((best_genome, best_fitness, fitness.calls))

  #   return bests1

  options = [option1, option2, option3, option4]

  for i, option in enumerate(options):


    for generation in range(MAX_GENERATIONS):
      fitness_values=[]
      for individual in population:
        key = tuple(individual)
        if key not in fitness_cache:
            fitness_cache[key] = fitness(individual)
        fitness_values.append(fitness_cache[key])

      fitness_scores = [fitness(individual) for individual in population]
      best_individual = population[fitness_scores.index(max(fitness_scores))]
      bests1 = option(population, fitness_scores)

    sorted_list = sorted(bests1, key=lambda x: x[2], reverse=False)
    print(f'OPTION {i+1}, number = {problem_instance_number}')
    (first_genome, fitness_first, calls_first) = sorted_list[0]
    print(f'Minimum number of calls = {calls_first} -> associated fitness = {round(fitness_first * 100, 2)}%')
    # just to check if the genome with the highest fitness is not the one with the lowest number of calls
    # a_sorted_list = sorted(bests1, key=lambda x: x[1], reverse=True)
    # (a_first_genome, a_fitness_first, a_calls_first) = a_sorted_list[0]
    # print(f'Maximum Fitness = {a_fitness_first} -> associated number of calls = {a_calls_first}')

  print('\n')



Parameters for this run:
mutation rate=0.001, crossover rate=0.5, population size=200, tournament size=7, max number of generations=10, offspring size=20
PROBLEM INSTANCE NUMBER = 1:
OPTION 1, number = 1
Minimum number of calls = 2558 -> associated fitness = 54.8%
OPTION 2, number = 1
Minimum number of calls = 4701 -> associated fitness = 54.2%
OPTION 3, number = 1
Minimum number of calls = 6711 -> associated fitness = 54.2%
OPTION 4, number = 1
Minimum number of calls = 8711 -> associated fitness = 54.2%


PROBLEM INSTANCE NUMBER = 2:
OPTION 1, number = 2
Minimum number of calls = 2522 -> associated fitness = 25.63%
OPTION 2, number = 2
Minimum number of calls = 4673 -> associated fitness = 25.61%
OPTION 3, number = 2
Minimum number of calls = 6684 -> associated fitness = 25.8%
OPTION 4, number = 2
Minimum number of calls = 8684 -> associated fitness = 25.8%


PROBLEM INSTANCE NUMBER = 5:
OPTION 1, number = 5
Minimum number of calls = 2559 -> associated fitness = 10.99%
OPTION 2, numb

Overall, the option1 is the one which behaves better in almost all the case and generically with the biggest part of the configurations. However, the option2 beats the option1 in the other cases (with also quite-high difference in fitness percentage).
Unfortunately, none of the options reach or overcome the 60% in any case.
Last consideration regards the fact that lower is the problem instance, better is the fitness.

Next step would be let the algorithm run with all the possible configurations to find the best one (i.e, the one that reach a percentage near to 100%)